# Maximum Independent Set
Quantum Optimization Benchmark Library: https://doi.org/10.48550/arXiv.2504.03832

Hardware: Apple M1 Pro (8 cores, 16 GB memory)

## Benchmark

In [2]:
using Quicopt, LinearAlgebra, Printf
using CPUTime
include("../../src/utils.jl")
Base.show(io::IO, f::Float64) = @printf(io, "%1.5f", f)

In [3]:
PATH = "../../instances/max_independent_set/"
filename = "brock400_1.gph";

In [4]:
# load graph from file
graph = load_dimacs_graph_matrix(PATH * filename)

# system size
N = size(graph)[1]

400

In [5]:
# number of edges
size(findall(x -> x==1, UpperTriangular(graph)), 1)

20077

In [ ]:
# Lagrange multiplier
λ = 0.5

# cost function
local_fields = -ones(N) ./ λ + 0.5 .* (ones(N)' * graph)';
couplings = -0.5 .* graph

tensor_problem = TensorProblem(local_fields, couplings);

In [7]:
# parameters
T = 200.0
tol = 1e-4;

In [8]:
# run once to force precompilation
mf_sol = solve(tensor_problem, T, atol=tol, rtol=tol)
sol_vec = sign.(mf_sol.u[end][3, :]);

In [9]:
# map from Ising to binary
bit_string = (1 .- sol_vec) ./ 2

# check violated clauses
num_violated_clauses = Int(0.5 * bit_string' * graph * bit_string)
num_violated_clauses |> println

# check max. indep. set
sum(bit_string) |> println

4
27.00000


In [10]:
false_idxs = findall(x -> x == 1., bit_string .* (graph * bit_string))
violated_bonds = Set([[idx, intersect(false_idxs, findall(x -> x == 1, graph[idx, :]))[1]] |> sort for idx in false_idxs])

# flip violated bonds
flip_idxs = [bond[1] for bond in violated_bonds]
bit_string[flip_idxs] .= 0.
bit_string' * graph * bit_string |> println
sum(bit_string) |> println

0.00000
23.00000


In [11]:
# get CPU time
@CPUtime begin
    mf_sol = solve(tensor_problem, T, atol=tol, rtol=tol)
    sol_vec = sign.(mf_sol.u[end][3, :])
    false_idxs = findall(x -> x == 1., bit_string .* (graph * bit_string))
    violated_bonds = Set([[idx, intersect(false_idxs, findall(x -> x == 1, graph[idx, :]))[1]] |> sort for idx in false_idxs])
    flip_idxs = [bond[1] for bond in violated_bonds]
    bit_string[flip_idxs] .= 0.    
end;

elapsed CPU time: 1.69669 seconds
